In [ ]:
import pandas as pd 
import numpy as np 
import os
import csv
import matplotlib.pyplot as plt
import glob

In [ ]:
#NRC
neighbor_1base=[(0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6), 
                        (5, 9), (9, 10), (10, 11), (9, 12), (12, 13), (9, 14), (9, 15), (9, 16), 
                        (6, 17), (17, 18), (18, 19), (17, 20), (20, 21), (17, 22), (17, 23), (17, 24)]

body_parts = [0, 1, 2, 5, 6, 7, 8, 9, 10,
        91, 93, 95, 96, 99, 103, 107, 111, 
        112, 114, 116, 117, 120, 124, 128, 132]

In [ ]:
#KIMORE
neighbor_1base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                                      (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                                      (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                                      (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                                      (22, 23), (23, 8), (24, 25), (25, 12)]

body_parts = ['index_Spine_Base', 'index_Spine_Mid', 'index_Neck', 'index_Head', 'index_Shoulder_Left', 
                      'index_Elbow_Left', 'index_Wrist_Left', 'index_Hand_Left', 'index_Shoulder_Right', 'index_Elbow_Right', 
                      'index_Wrist_Right', 'index_Hand_Right', 'index_Hip_Left', 
                      
                      
                      
                      
                      'index_Knee_Left', 'index_Ankle_Left', 
                     'index_Foot_Left', 'index_Hip_Right', 'index_Knee_Right', 'index_Ankle_Right', 'index_Ankle_Right', 'index_Spine_Shoulder', 
                      'index_Tip_Left', 'index_Thumb_Left', 'index_Tip_Right', 'index_Thumb_Right'
                      ]

## Check The Datasets
please change to your dataset

In [ ]:
sample = np.load('/dataset/NRC_skeleton/npy_withz_segment/ND18_day1_session7_label4.npy')
sample.shape #(155, 532)
sample = sample.reshape(155, 133, 4)

(155, 532)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display
import copy

for i in aframe:
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(i[:25,0], i[:25,1], i[:25,2])

        for l in neighbor_1base:
                side = body_parts[l[0]-1]
                side = side.split('_')[-1]
                if side == 'Left':
                        color = 'blue'
                elif side == 'Right':
                        color = 'red'
                else:
                        color = 'green'
                ax.plot([i[l[0]-1, 0],i[l[1]-1, 0]], [i[l[0]-1,1],i[l[1]-1,1]], [i[l[0]-1,2], i[l[1]-1,2]], c=color)

        ax.view_init(150,60)
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.show()
        plt.cla()

### this is KIMORE & rehab model lab

In [ ]:
import my_rehab.sgcn_lstm
from my_rehab.sgcn_lstm import *
from my_rehab.data_processing import Data_set, NRC_Data_set
from my_rehab.graph import Graph
import STGCN_rehab.Data
from STGCN_rehab.GCN import data_processing as dp

from torch.utils.data import DataLoader
torch_data = NRC_Data_set('/home/neuronS1/NRC/NRC_rehab_train_ST_02.csv')
orin_data = dp.Data_Loader('/dataset2/KIMORE/CG/Expert/E_ID4/Es5/Raw/JointPosition021214_123055.csv', '/home/neuronS1/NRC/Train_Y.csv')
 
graph = Graph(len(torch_data.body_part))
traindataset = DataLoader(torch_data, batch_size=1)
model = train_network(AD=graph.AD, AD2=graph.AD2, bias_mat_1=graph.bias_mat_1, bias_mat_2=graph.bias_mat_2)

/home/neuronS1/anaconda3/envs/NRC/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'preprocess'

### test pretrain mode - for tensorflow ver.

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)
new_model = keras.models.load_model('/home/neuronS1/NRC/STGCN_rehab/pretrain model/best_model.hdf5', custom_objects={'tf': tf})

2.4.0


In [ ]:
import STGCN_rehab.Data
from STGCN_rehab.GCN import data_processing as dp
import numpy as np

In [ ]:
orin_data = dp.Data_Loader('/dataset2/KIMORE/CG/Expert/E_ID4/Es5/Raw/JointPosition021214_123055.csv', '/home/neuronS1/NRC/Train_Y.csv')

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)

### jerk & rehab model label

##### 1. check available

In [ ]:
before_path = '/dataset2/nrc-main-02/data-preprocessed/export/imu_segment/2Stroke03_day1_session3_label08.csv'
after_path = '/dataset2/nrc-main-02/data-preprocessed/export/imu_segment/2Stroke03_day4_session3_label08.csv'

In [ ]:
data_df = pd.read_csv(before_path, index_col=0)
data_df.reset_index(inplace=True, drop=True)

data_accs = [col for col in data_df.columns if 'Acc' in col]
data_jerkdf = pd.DataFrame(columns=data_accs, index=data_df.index)

for idx in range(len(data_df) - 1):
    for col in data_jerkdf.columns:
        data_jerkdf.loc[idx, col] = data_df.loc[idx+1, col] - data_df.loc[idx, col]

data_med = []
for col in data_jerkdf.columns:
    data_med.append(data_jerkdf.loc[:, col].abs().median())
data_med = np.array(data_med)
jerk_score = max(data_med[:3].mean(), data_med[3:].mean())

print("before_data: ", jerk_score)

In [ ]:
data_df = pd.read_csv(after_path, index_col=0)
data_df.reset_index(inplace=True, drop=True)

data_accs = [col for col in data_df.columns if 'Acc' in col]
data_jerkdf = pd.DataFrame(columns=data_accs, index=data_df.index)

for idx in range(len(data_df) - 1):
    for col in data_jerkdf.columns:
        data_jerkdf.loc[idx, col] = data_df.loc[idx+1, col] - data_df.loc[idx, col]

data_med = []
for col in data_jerkdf.columns:
    data_med.append(data_jerkdf.loc[:, col].abs().median())
data_med = np.array(data_med)
jerk_score = max(data_med[:3].mean(), data_med[3:].mean())

print("after_data: ", jerk_score)

##### 2. make rehab- label file

In [39]:
jerk_list = glob.glob('/dataset2/nrc-main02/data-preprocessed/export/imu_segment/'+"*.csv")

In [96]:
import csv
f1 = open("NRC_rehab_train_ST_02.csv", "w")
Stroke = csv.writer(f1)

f2 = open("NRC_rehab_train_ND_02.csv", "w")
Nd = csv.writer(f2)

In [86]:
import glob
dir = '/dataset2/nrc-main02/estimated-json/npy_z/'
skeleton_list = glob.glob(dir+"*")

In [87]:
c1 = 0
c2=0
c3=0

In [88]:
len(jerk_list), len(skeleton_list)

(5940, 6301)

In [32]:
data_df = pd.read_csv(jerk_list[0], index_col=0)
data_df

,datetime,L_Acc_X,L_Acc_Y,L_Acc_Z,L_Gyr_X,L_Gyr_Y,L_Gyr_Z,R_Acc_X,R_Acc_Y,R_Acc_Z,R_Gyr_X,R_Gyr_Y,R_Gyr_Z,session,label
27,2024-10-18 12:47:33.369474,-7.614986,-3.363549,5.053604,-2.896855,-1.275225,-3.634705,3.412898,-2.110647,6.326813,-51.970947,-10.028509,40.603363,4,2
28,2024-10-18 12:47:33.386141,-7.719246,-3.389643,5.067219,-2.280740,-1.824489,-3.013891,3.609621,-2.002658,6.460279,-38.076256,-12.117304,43.225109,4,2
29,2024-10-18 12:47:33.402808,-7.773113,-3.420020,5.146438,-1.463081,-2.356618,-2.834078,3.849570,-1.717398,6.762733,-31.689192,-11.511996,46.275845,4,2
30,2024-10-18 12:47:33.419475,-7.737568,-3.364017,5.176956,-0.776327,-2.994900,-2.600016,3.622660,-1.887765,7.137947,-39.567940,-12.656790,44.653145,4,2
31,2024-10-18 12:47:33.436142,-7.667928,-3.307122,5.217874,-1.137658,-3.735568,-2.023743,3.209963,-2.720409,7.491507,-44.855339,-18.140959,40.979309,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,2024-10-18 12:47:42.102982,-7.773157,-3.151994,5.160930,5.157829,0.872606,0.108617,-1.274383,2.140020,9.553386,13.534707,-4.880151,2.470475,4,2
552,2024-10-18 12:47:42.119649,-7.750852,-3.149096,5.184074,3.964641,0.544027,-0.154758,-1.153973,2.775920,9.530676,15.520220,-3.410789,1.029529,4,2
559,2024-10-18 12:47:42.236318,-7.703014,-3.142288,5.390872,2.153721,-2.188727,2.250627,-1.629849,2.152736,9.462541,-9.678972,-5.033787,0.231308,4,2
562,2024-10-18 12:47:42.286319,-7.712762,-3.205992,5.114915,0.419049,-1.961465,1.139637,-1.291290,2.645359,9.461684,-3.301575,-9.800444,8.269218,4,2


In [97]:
for jl_path in jerk_list:
    i = jl_path.split('/')[-1][:-3]
    
    if i in non:
        continue

    name = glob.glob(dir+i+'npy')
    if len(name) < 1: #only imu
        continue
    else:#?????
        name = name[0]
    data_df = pd.read_csv(jl_path, index_col=0)
    data_df.reset_index(inplace=True, drop=True)

    data_accs = [col for col in data_df.columns if 'Acc' in col]
    data_jerkdf = pd.DataFrame(columns=data_accs, index=data_df.index)

    for idx in range(len(data_df) - 1):
        for col in data_jerkdf.columns:
            data_jerkdf.loc[idx, col] = data_df.loc[idx+1, col] - data_df.loc[idx, col]

    data_med = []
    for col in data_jerkdf.columns:
        data_med.append(data_jerkdf.loc[:, col].abs().median())
    data_med = np.array(data_med)
    jerk_score = max(data_med[:3].mean(), data_med[3:].mean())

    if np.isnan(jerk_score):
        continue

    if name.find('ND') == -1:#stroke
        Stroke.writerow([name,jerk_score])
    else:
        Nd.writerow([name,jerk_score])


f1.close()
f2.close()

#### there is the file but inside is empthy

In [67]:
f2 = open("NRC_rehab_train_ND_02.csv", "r")
Nd = csv.reader(f2)

non = []
for idx, i in enumerate(Nd):
    if i[1] == 'nan':
        non.append(i[0].split('/')[-1][:-3])

In [70]:
for idx, i in enumerate(non):
    get_index = jerk_list.index('/dataset2/nrc-main02/data-preprocessed/export/imu_segment/'+i+'csv')

    print(idx, jerk_list[get_index])
    data_df = pd.read_csv(jerk_list[get_index], index_col=0)
    print(data_df)

0 /dataset2/nrc-main02/data-preprocessed/export/imu_segment/2ND08_session1_label15.csv
Empty DataFrame
Columns: [datetime, L_Acc_X, L_Acc_Y, L_Acc_Z, L_Gyr_X, L_Gyr_Y, L_Gyr_Z, R_Acc_X, R_Acc_Y, R_Acc_Z, R_Gyr_X, R_Gyr_Y, R_Gyr_Z, session, label]
Index: []
1 /dataset2/nrc-main02/data-preprocessed/export/imu_segment/2ND08_session1_label08.csv
Empty DataFrame
Columns: [datetime, L_Acc_X, L_Acc_Y, L_Acc_Z, L_Gyr_X, L_Gyr_Y, L_Gyr_Z, R_Acc_X, R_Acc_Y, R_Acc_Z, R_Gyr_X, R_Gyr_Y, R_Gyr_Z, session, label]
Index: []
2 /dataset2/nrc-main02/data-preprocessed/export/imu_segment/2ND08_session1_label14.csv
Empty DataFrame
Columns: [datetime, L_Acc_X, L_Acc_Y, L_Acc_Z, L_Gyr_X, L_Gyr_Y, L_Gyr_Z, R_Acc_X, R_Acc_Y, R_Acc_Z, R_Gyr_X, R_Gyr_Y, R_Gyr_Z, session, label]
Index: []
3 /dataset2/nrc-main02/data-preprocessed/export/imu_segment/2ND08_session1_label06.csv
Empty DataFrame
Columns: [datetime, L_Acc_X, L_Acc_Y, L_Acc_Z, L_Gyr_X, L_Gyr_Y, L_Gyr_Z, R_Acc_X, R_Acc_Y, R_Acc_Z, R_Gyr_X, R_Gyr_Y, R_Gyr_Z

In [ ]:
from STGCN_rehab.GCN import *
import tensorflow as tf
from tensorflow import keras

In [ ]:
algorithm = keras.models.load_model('/home/neuronS1/NRC/STGCN_rehab/pretrain model/_best_model.hdf5', custom_objects={'tf': tf})

/home/neuronS1/anaconda3/envs/NRC-rehab/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1059: UserWarning: sgcn_lstm is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)
2025-03-20 19:52:10.777508: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-03-20 19:52:10.778317: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-03-20 19:52:10.783670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: NVIDIA GeForce RTX 4090 computeCapability: 8.9
coreClock: 2.52GHz coreCount: 128 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 938.86GiB/s
2025-03-20 19:52:10.783802: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:68:00.0 name: NVIDIA GeForce RTX 4090 computeCapability: 8.9
coreClock: 2.52GHz cor

In [18]:
_, test_y = data_loader.__getitem__(0)
test_x = np.expand_dims(data_loader.scaled_x[0], axis=0)

In [23]:
text_x = orin_data.scaled_x
test_y = orin_data.scaled_y

In [28]:
orin_data.train_y

array([[48.333]])

In [30]:
pred_y = algorithm.predict(test_x)
pred_y = orin_data.sc2.inverse_transform(pred_y)
print(test_y, pred_y)

[[0.]] [[47.80596]]


In [2]:
import STGCN_rehab.Data
from STGCN_rehab.GCN import data_processing as dp

In [3]:
data_loader = dp.NRC_Data_Loader('/home/neuronS1/NRC/NRC_rehab_train_ND_01.csv')

In [10]:
orin_data = dp.Data_Loader('/dataset2/KIMORE/CG/Expert/E_ID4/Es5/Raw/JointPosition021214_123055.csv', '/home/neuronS1/NRC/Train_Y.csv')

In [11]:
orin_data.scaled_x.shape

(1, 100, 25, 3)

In [13]:
data_loader.scaled_x.shape

(1548, 283, 25, 3)

In [2]:
import preprocess

ModuleNotFoundError: No module named 'preprocess'

In [1]:
import my_rehab.sgcn_lstm
from my_rehab.sgcn_lstm import *
from my_rehab.data_processing import Data_set, NRC_Data_set
from my_rehab.graph import Graph

import numpy as np

/home/neuronS1/anaconda3/envs/NRC/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trainDataset = NRC_Data_set('/home/neuronS1/NRC/NRC_rehab_train_ND_01.csv')  # folder name -> Train.csv, Test.csv

 Load DATA 1547 / 1548